This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run

Dependencies:

    pymatgen(latest development version from github)
    Polymers
    packmol

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pprint import pprint

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.packmol import PackmolRunner
from polymers.utils import get_topology

# Set the start, bulk, and terminal molecules. Set charges and optionally the forcefield species name mapping 

In [2]:
# start molecule
peo_start = Molecule.from_file("peo_start.xyz")
s_charges = [-0.1187, 0.0861, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_start.add_site_property("charge", s_charges)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("peo_bulk.xyz")
b_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.0326, 0.0861, 0.0861]
peo_bulk.add_site_property("charge", b_charges)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("peo_end.xyz")
e_charges = [-0.0326, 0.0861, 0.0861, -0.2792, -0.1187, 0.0861, 0.0861, 0.0861]
peo_end.add_site_property("charge", e_charges)
e_head = 0
e_tail = 4

# Create polymer via random walk

In [3]:
n_units = 58
link_distance = 1.5075

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

# linear chain
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                             peo_bulk, head, tail, 
                             peo_end, e_head, e_tail, 
                             n_units, link_distance, linear_chain=True)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")
#peo_polymer_linear.molecule.to(filename="polymer_linear.xyz", fmt="xyz")

# Set the topology of the polymer from the linear chain

In [4]:
from pymatgen.io.lammps.topology import Topology

topology = Topology.from_molecule(peo_polymer_linear.molecule)

natoms, nbonds, nangles, ndih =  len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)

print(natoms, nbonds, nangles, ndih)

(408, 407, 754, 861)


# Generate polymer matrix

In [5]:
#Polymer matrix settings

# constituent molecules
molecules = [peo_polymer.molecule]
matrix_config = [{"number": 1, "inside box":[0,0,0,100,100,100]}]

# Use Packmol to pack the polymer chains into a box

In [6]:
pmr = PackmolRunner(molecules,
                    matrix_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz


# Read in the force field parameters

In [7]:
from pymatgen.io.lammps.force_field import ForceField

forcefield = ForceField.from_file("ff_data.yaml")
#print(forcefield.bonds)

# Generate lammps data file from the topology and the forcefield

In [8]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols_number = [mol_config["number"] for mol_config in matrix_config]
box_size = [[0.0, 150], 
            [0.0, 150], 
            [0.0, 150]]

# list of molecular topologies for each constituent molecules
topologies = [topology]*len(molecules)

lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(molecules, mols_number, 
                                                                   box_size, packed_polymer, 
                                                                   forcefield, topologies)

# Write the data file

In [9]:
print str(lammps_ff_data)

#write the data file
lammps_ff_data.write_data_file("lammps_data.dat")

Data file generated by pymatgen

408 atoms
407 bonds
754 angles
861 dihedrals

3 atom types
3 bond types
5 angle types
5 dihedral types

0.0 150 xlo xhi
0.0 150 ylo yhi
0.0 150 zlo zhi

Masses 

1 1.00794
2 12.0107
3 15.9994

Bond Coeffs 

1 1000 1.4115
2 1000 1.1041
3 1000 1.5075

Angle Coeffs 

1 42.9 110.1
2 38.5 109.47
3 56.0 109.48
4 86.0 108.54
5 74.5 108.05

Dihedral Coeffs 

1 0.0 0.0 -0.73 0.0
2 0.0 0.0 0.28 0.0
3 1.76 0.67 0.04 0.0
4 0.0 0.0 0.28 0.0
5 0.41 -2.1 -0.6 -0.82

Atoms 

1 1 2 -0.1187 57.8763153254 95.1789532489 71.1981909518
2 1 1 0.0861 57.4931583254 96.1472082489 70.9520669518
3 1 1 0.0861 57.5327243254 94.4494042489 70.4440429518
4 1 1 0.0861 57.5033733254 94.8826852489 72.1942709518
5 1 3 -0.2792 59.3830883254 95.2235362489 71.2124219518
6 1 2 -0.0326 59.8557043254 93.9343592489 71.5394929518
7 1 1 0.0861 59.5121143254 93.2048102489 70.7853449518
8 1 1 0.0861 59.4827633254 93.6380912489 72.5355729518
9 1 2 -0.0326 61.3624783254 93.9789422489 71.5537239518
10 1

# Setup input file

In [14]:
from pymatgen.io.lammps.input import DictLammpsInput

lammps_input = DictLammpsInput.from_file("NPT","NPT.json")
lammps_input.write_input("lammps.in")
print str(lammps_input)

units  real
dimension  3
atom_style  full
boundary  p p p
neighbor  10.0 bin
bond_style  harmonic
angle_style  harmonic
dihedral_style  opls
read_data  in.data
pair_style  buck/coul/cut 15.0
pair_coeff 1 1 2649.6 0.2674 27.22
pair_coeff 1 2 4320.0 0.2928 137.6
pair_coeff 1 3 14176.0 0.2563 104.0
pair_coeff 2 2 14976.0 0.3236 637.6
pair_coeff 2 3 33702.4 0.2796 503.0
pair_coeff 3 3 75844.8 0.2461 396.9
dump DUMP all custom 200 poly.lammpstrj id type x y z ix iy iz mol
dump traj all dcd 100 lammps_traj.dcd
thermo_style  custom step temp press etotal fmax fnorm
thermo  100
fix  NPT all npt temp 298 298 100.0 iso 0.0 0.0 100.0
run  1000



# Run Lammps

In [15]:
!mpiexec -n 4 lammps -in lammps.in

LAMMPS (10 Feb 2015)
Reading data file ...
  orthogonal box = (0 0 0) to (150 150 150)
  1 by 2 by 2 MPI processor grid
  reading atoms ...
  408 atoms
  scanning bonds ...
  4 = max bonds/atom
  scanning angles ...
  6 = max angles/atom
  scanning dihedrals ...
  9 = max dihedrals/atom
  reading bonds ...
  407 bonds
  reading angles ...
  754 angles
  reading dihedrals ...
  861 dihedrals
Finding 1-2 1-3 1-4 neighbors ...
  4 = max # of 1-2 neighbors
  6 = max # of 1-3 neighbors
  12 = max # of 1-4 neighbors
  14 = max # of special neighbors
Neighbor list info ...
  1 neighbor list requests
  update every 1 steps, delay 10 steps, check yes
  master list distance cutoff = 25
Setting up run ...
Memory usage per processor = 12.3891 Mbytes
Step Temp Press TotEng Fmax Fnorm 
       0            0   -101.56733    8611.3393    296.58439    2687.9698 
     100          nan          nan          nan          nan          nan 
     200          nan          nan          nan          nan       

# Generate VMD structure and load trajectory

requires vmd with topotools package

In [43]:
%%bash
#this path is machine specific
/Applications/VMD\ 1.9.2.app/Contents/Resources/VMD.app/Contents/MacOS/VMD -dispdev win << HERE
#tcl script
package require topotools
topo readlammpsdata lammps_data.dat full
#animate write psf lammps_data.psf
mol addfile lammps_traj.dcd type dcd waitfor all
HERE

Info) VMD for MACOSXX86, version 1.9.2 (December 29, 2014)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 8 CPUs detected.
Info) OpenGL renderer: Intel Iris Pro OpenGL Engine
Info)   Features: STENCIL MDE MTX NPOT PP PS GLSL(OVF) 
Info)   Full GLSL rendering mode is available.
Info)   Textures: 2-D (16384x16384), 3-D (2048x2048x2048), Multitexture (8)
Info) Dynamically loaded 2 plugins in directory:
Info) /Applications/VMD 1.9.2.app/Contents/vmd/plugins/MACOSXX86/molfile
Info) VMD for MACOSXX86, version 1.9.2 (December 29, 2014)
Info) Exiting normally.
/bin/sh: package: command not found
/bin/sh: top